In [2]:
from zipfile import ZipFile
f = ZipFile("/content/drive/MyDrive/chinese_news_trans.zip")
f.extractall()

f = ZipFile("/content/drive/MyDrive/chinese_news_test.zip")
f.extractall()

In [7]:
# glob(路近的path) *任意
import glob
import os
import pandas as pd

def process(base):
  datas = {
      "contens":[],
      "ans":[]
  }
  dirs = glob.glob(os.path.join(base, "*"))
  for d in dirs:
    fns1 = os.path.join(d, "*.txt")
    fns2 = os.path.join(d, "*.TXT")
    fns = glob.glob(fns1) + glob.glob(fns2)
    for fn in fns:
      with open(fn, "r", encoding="utf-8") as f:
        datas["contens"].append(f.read())
      datas["ans"].append(os.path.split(d)[-1])
  df = pd.DataFrame(datas)
  return df

train_df = process("chinese_news_trans")
test_df = process("chinese_news_test")

In [ ]:
u = train_df["ans"].unique()
name2cat = {name:i for i, name in enumerate(u)}
cat2name = {i:name for i, name in enumerate(u)}

y_train = train_df["ans"].replace(name2cat)
y_test = test_df["ans"].replace(name2cat)
y_train

In [ ]:
import jieba
from urllib.request import urlretrieve

url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")

jieba.set_dictionary("dict.txt.big")
def news(n):
  return " ".join(jieba.cut(n))
x_train = train_df["contens"].apply(news)
x_test = test_df["contens"].apply(news)

x_test

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
pre = clf.predict(x_test_count)
print(accuracy_score(y_test, pre))
prob = clf.predict_proba(x_test_count)
print(log_loss(y_test, prob))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      index=[name + "(原本)" for name in u],
      columns=[name + "(預測)" for name in u])

In [ ]:
p = input("write:")

test = vec.transform([news(p)])
prob = clf.predict_proba(test)[0]
for i in range(len(u)):
  print(u[i],round(prob[i], 3))

In [ ]:
!pip install opencc-python-reimplemented

In [ ]:
from opencc import OpenCC
cc = OpenCC('s2twp')  # convert from Simplified Chinese to Traditional Chinese
# can also set conversion by calling set_conversion
# cc.set_conversion('s2tw')
to_convert = '开放中文转换'
converted = cc.convert(to_convert)
converted

'開放中文轉換'